In [96]:
import findspark

In [97]:
findspark.init()

In [98]:
from pyspark.sql import SparkSession

In [99]:
spark = SparkSession.builder.getOrCreate()

In [100]:
spark

In [101]:
sc = spark.sparkContext

In [102]:
 # Task 
dataRDD = sc.parallelize([('Brooke',20),('Denny',31),('Jules',30),('TD',35),
                          ('Brooke',25),('Jules',40),('Denny',51),('TD',15),
                          ('TD',12)])


In [103]:
dataRDD.collect() # data written in list 

[('Brooke', 20),
 ('Denny', 31),
 ('Jules', 30),
 ('TD', 35),
 ('Brooke', 25),
 ('Jules', 40),
 ('Denny', 51),
 ('TD', 15),
 ('TD', 12)]

In [104]:
data_df = spark.createDataFrame(dataRDD, ['name','age'])

In [105]:
data_df.printSchema()

root
 |-- name: string (nullable = true)
 |-- age: long (nullable = true)



In [106]:
 data_df.show()

+------+---+
|  name|age|
+------+---+
|Brooke| 20|
| Denny| 31|
| Jules| 30|
|    TD| 35|
|Brooke| 25|
| Jules| 40|
| Denny| 51|
|    TD| 15|
|    TD| 12|
+------+---+



In [107]:
data_df.show(3)

+------+---+
|  name|age|
+------+---+
|Brooke| 20|
| Denny| 31|
| Jules| 30|
+------+---+
only showing top 3 rows


In [108]:
data_df.collect()

[Row(name='Brooke', age=20),
 Row(name='Denny', age=31),
 Row(name='Jules', age=30),
 Row(name='TD', age=35),
 Row(name='Brooke', age=25),
 Row(name='Jules', age=40),
 Row(name='Denny', age=51),
 Row(name='TD', age=15),
 Row(name='TD', age=12)]

In [109]:
r = data_df.collect()[0]

In [110]:
r.name

'Brooke'

In [111]:
r.age

20

In [112]:
data_df.take(5)

[Row(name='Brooke', age=20),
 Row(name='Denny', age=31),
 Row(name='Jules', age=30),
 Row(name='TD', age=35),
 Row(name='Brooke', age=25)]

In [113]:
avg_age =data_df.groupby('name').avg('age')

In [114]:
avg_age.show()

+------+------------------+
|  name|          avg(age)|
+------+------------------+
|Brooke|              22.5|
| Denny|              41.0|
| Jules|              35.0|
|    TD|20.666666666666668|
+------+------------------+



In [115]:
avg_age.rdd.getNumPartitions()

1

In [116]:
data_df.rdd.getNumPartitions()

4

In [117]:
data_df.head()

Row(name='Brooke', age=20)

In [118]:
from pyspark.sql.functions import avg

In [119]:
 avg_age =data_df.groupby('name').agg(avg('age').alias('Average age'))

In [120]:
avg_age.printSchema()

root
 |-- name: string (nullable = true)
 |-- Average age: double (nullable = true)



In [121]:
avg_age.show()

+------+------------------+
|  name|       Average age|
+------+------------------+
|Brooke|              22.5|
| Denny|              41.0|
| Jules|              35.0|
|    TD|20.666666666666668|
+------+------------------+



In [122]:
df = spark.read.json('people.json')

In [123]:
df.printSchema()

root
 |-- age: long (nullable = true)
 |-- name: string (nullable = true)



In [124]:
df.show()

+----+-------+
| age|   name|
+----+-------+
|NULL|Michael|
|  30|   Andy|
|  19| Justin|
+----+-------+



In [125]:
df.age , df.name

(Column<'age'>, Column<'name'>)

In [126]:
df['age'] , df['name']

(Column<'age'>, Column<'name'>)

In [127]:
from pyspark.sql.functions import col

In [128]:
col('AAA')

Column<'AAA'>

In [129]:
df.select('name')

DataFrame[name: string]

In [130]:
df_names =df.select('name')

In [131]:
df_names.printSchema()

root
 |-- name: string (nullable = true)



In [132]:
df_names.show()

+-------+
|   name|
+-------+
|Michael|
|   Andy|
| Justin|
+-------+



In [133]:
df_ages = df.select("age")

In [134]:
df_ages.show()

+----+
| age|
+----+
|NULL|
|  30|
|  19|
+----+



In [135]:
# Spark Job is A collection of Transformation followed By Action 

In [136]:
df_adults =df_ages.where(col('age') > 20)

In [137]:
df_adults.show()

+---+
|age|
+---+
| 30|
+---+



In [138]:
# OPerations in DF in 'DAG'

In [139]:
df.cache() 

25/10/16 00:56:52 WARN CacheManager: Asked to cache already cached data.


DataFrame[age: bigint, name: string]

In [140]:
#-----------------------------------------------------

In [141]:
import pandas as pd 

In [142]:
pdf = pd.read_csv('fruits.csv')

In [143]:
df = spark.createDataFrame(pdf)
df.show()

+------+------+---+---+
| color| fruit| v1| v2|
+------+------+---+---+
|   red| apple|3.0|5.0|
|   red| apple|5.0|6.0|
|Yellow| banna|5.0|3.0|
|  blue| grape|6.0|1.0|
|Yellow| banna|2.0|7.0|
|  blue| grape|7.0|1.0|
|Yellow| banna|1.0|2.0|
|Yellow|carrot|2.0|3.0|
|  blue| grape|7.0|1.0|
|Yellow| banna|1.0|2.0|
+------+------+---+---+



In [144]:
df.cache()

DataFrame[color: string, fruit: string, v1: double, v2: double]

In [145]:
df.groupBy('fruit').avg().show()

+------+-----------------+-------+
| fruit|          avg(v1)|avg(v2)|
+------+-----------------+-------+
| apple|              4.0|    5.5|
| grape|6.666666666666667|    1.0|
| banna|             2.25|    3.5|
|carrot|              2.0|    3.0|
+------+-----------------+-------+



In [146]:
df.groupBy('fruit').sum().show()

+------+-------+-------+
| fruit|sum(v1)|sum(v2)|
+------+-------+-------+
| apple|    8.0|   11.0|
| grape|   20.0|    3.0|
| banna|    9.0|   14.0|
|carrot|    2.0|    3.0|
+------+-------+-------+



In [147]:
df.groupBy('fruit').sum('v1').show()

+------+-------+
| fruit|sum(v1)|
+------+-------+
| apple|    8.0|
| grape|   20.0|
| banna|    9.0|
|carrot|    2.0|
+------+-------+



In [148]:
df.groupBy('fruit').agg(avg('v1').alias('avg_v1')).show()

+------+-----------------+
| fruit|           avg_v1|
+------+-----------------+
| apple|              4.0|
| grape|6.666666666666667|
| banna|             2.25|
|carrot|              2.0|
+------+-----------------+



In [149]:
from pyspark.sql.functions import *
import pyspark.sql.functions as fn 

In [150]:
df.groupBy('color').sum('v1').show()

+------+-------+
| color|sum(v1)|
+------+-------+
|   red|    8.0|
|Yellow|   11.0|
|  blue|   20.0|
+------+-------+



In [151]:
  df.groupBy('color','fruit').sum('v1','v2').show()

+------+------+-------+-------+
| color| fruit|sum(v1)|sum(v2)|
+------+------+-------+-------+
|   red| apple|    8.0|   11.0|
|  blue| grape|   20.0|    3.0|
|Yellow| banna|    9.0|   14.0|
|Yellow|carrot|    2.0|    3.0|
+------+------+-------+-------+



In [152]:
df.groupBy('color').agg(avg('v1').alias('average_v1')).show()

+------+-----------------+
| color|       average_v1|
+------+-----------------+
|   red|              4.0|
|Yellow|              2.2|
|  blue|6.666666666666667|
+------+-----------------+



In [153]:
df.groupBy('color').agg(avg('v1').alias('avg_v1') , avg('v2').alias('avg_v2')).show()

+------+-----------------+------+
| color|           avg_v1|avg_v2|
+------+-----------------+------+
|   red|              4.0|   5.5|
|Yellow|              2.2|   3.4|
|  blue|6.666666666666667|   1.0|
+------+-----------------+------+



In [154]:
df.groupBy('color','fruit').agg(avg('v1').alias('avg_v1') , avg('v2').alias('avg_v2') ,fn.sum('v1')).show()

+------+------+-----------------+------+-------+
| color| fruit|           avg_v1|avg_v2|sum(v1)|
+------+------+-----------------+------+-------+
|   red| apple|              4.0|   5.5|    8.0|
|  blue| grape|6.666666666666667|   1.0|   20.0|
|Yellow| banna|             2.25|   3.5|    9.0|
|Yellow|carrot|              2.0|   3.0|    2.0|
+------+------+-----------------+------+-------+



In [155]:
df.persist() #after first action , df will persisted in memory

25/10/16 00:56:56 WARN CacheManager: Asked to cache already cached data.


DataFrame[color: string, fruit: string, v1: double, v2: double]

In [156]:
df.groupBy('color','fruit').agg(count('fruit').alias('#fruits')).show()

+------+------+-------+
| color| fruit|#fruits|
+------+------+-------+
|   red| apple|      2|
|  blue| grape|      3|
|Yellow| banna|      4|
|Yellow|carrot|      1|
+------+------+-------+



In [157]:
df.unpersist() # remove df from memory

DataFrame[color: string, fruit: string, v1: double, v2: double]

In [158]:
df.cache()

DataFrame[color: string, fruit: string, v1: double, v2: double]

In [159]:
df_sorted = df.sort('fruit')

In [160]:
df_sorted.show()

+------+------+---+---+
| color| fruit| v1| v2|
+------+------+---+---+
|   red| apple|3.0|5.0|
|   red| apple|5.0|6.0|
|Yellow| banna|5.0|3.0|
|Yellow| banna|2.0|7.0|
|Yellow| banna|1.0|2.0|
|Yellow| banna|1.0|2.0|
|Yellow|carrot|2.0|3.0|
|  blue| grape|6.0|1.0|
|  blue| grape|7.0|1.0|
|  blue| grape|7.0|1.0|
+------+------+---+---+



In [161]:
df_sorted_2 = df.sort('color' , 'fruit')

In [162]:
df_sorted_2.show()

+------+------+---+---+
| color| fruit| v1| v2|
+------+------+---+---+
|Yellow| banna|1.0|2.0|
|Yellow| banna|5.0|3.0|
|Yellow| banna|2.0|7.0|
|Yellow| banna|1.0|2.0|
|Yellow|carrot|2.0|3.0|
|  blue| grape|6.0|1.0|
|  blue| grape|7.0|1.0|
|  blue| grape|7.0|1.0|
|   red| apple|3.0|5.0|
|   red| apple|5.0|6.0|
+------+------+---+---+



In [163]:
df_sorted_2.printSchema()

root
 |-- color: string (nullable = true)
 |-- fruit: string (nullable = true)
 |-- v1: double (nullable = true)
 |-- v2: double (nullable = true)



In [164]:
df.printSchema()

root
 |-- color: string (nullable = true)
 |-- fruit: string (nullable = true)
 |-- v1: double (nullable = true)
 |-- v2: double (nullable = true)



In [165]:
#Defining Schema for df
# Note : Parquet file has built in Schema
# -- Defining Schema as two types: (Programmatically , DDL)

In [166]:
#Schema Programmatically

from pyspark.sql.types import *

In [167]:
MySchema = StructType([StructField('author',StringType(),False),
                       StructField('title',StringType(),False),
                       StructField('pages',IntegerType(),False)])

In [168]:
# spark.read.csv(schema=MySchema)

In [169]:
# spark.createDataFrame(schema=MySchema)

In [170]:
MyNewSchema = 'ID INT not null, First STRING, Last STRING, Url STRING, Published STRING, Hits INT, Campaigns ARRAY<STRING>'

In [171]:
MyNewSchema_ = 'ID INT not null, First STRING, `Last Name` STRING, Url STRING, Published STRING, Hits INT, Campaigns ARRAY<STRING>'

In [172]:
MyNewSchema

'ID INT not null, First STRING, Last STRING, Url STRING, Published STRING, Hits INT, Campaigns ARRAY<STRING>'

In [173]:
data =[[1,"Jules","Verne","http://www.example.com/1","2014-01-01",254,["email","web"]],
       [2,"H.G.","Wells","http://www.example.com/2","2014-01-02",123,["web"]],
       [3,"Isaac","Asimov","http://www.example.com/3","2014-02-01",456,["email","ads","web"]],
       [4,"Douglas","Adams","http://www.example.com/4","2014-03-05",321,["ads","web"]],
       [5,"Arthur C.","Clarke","http://www.example.com/5","2014-04-01",213,["email","ads"]],
       [6,"Frank","Herbert","http://www.example.com/6","2014-04-15",312,["email","web"]],
       [7,"Ursula K.","Le Guin","http://www.example.com/7","2014-05-21",213,["web"]]]

In [174]:
blogs_df = spark.createDataFrame(data, schema = MyNewSchema)

In [175]:
blogs_df.show()

+---+---------+-------+--------------------+----------+----+-----------------+
| ID|    First|   Last|                 Url| Published|Hits|        Campaigns|
+---+---------+-------+--------------------+----------+----+-----------------+
|  1|    Jules|  Verne|http://www.exampl...|2014-01-01| 254|     [email, web]|
|  2|     H.G.|  Wells|http://www.exampl...|2014-01-02| 123|            [web]|
|  3|    Isaac| Asimov|http://www.exampl...|2014-02-01| 456|[email, ads, web]|
|  4|  Douglas|  Adams|http://www.exampl...|2014-03-05| 321|       [ads, web]|
|  5|Arthur C.| Clarke|http://www.exampl...|2014-04-01| 213|     [email, ads]|
|  6|    Frank|Herbert|http://www.exampl...|2014-04-15| 312|     [email, web]|
|  7|Ursula K.|Le Guin|http://www.exampl...|2014-05-21| 213|            [web]|
+---+---------+-------+--------------------+----------+----+-----------------+



In [176]:
blogs_df.printSchema()

root
 |-- ID: integer (nullable = false)
 |-- First: string (nullable = true)
 |-- Last: string (nullable = true)
 |-- Url: string (nullable = true)
 |-- Published: string (nullable = true)
 |-- Hits: integer (nullable = true)
 |-- Campaigns: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [177]:
blogs_df.show()

+---+---------+-------+--------------------+----------+----+-----------------+
| ID|    First|   Last|                 Url| Published|Hits|        Campaigns|
+---+---------+-------+--------------------+----------+----+-----------------+
|  1|    Jules|  Verne|http://www.exampl...|2014-01-01| 254|     [email, web]|
|  2|     H.G.|  Wells|http://www.exampl...|2014-01-02| 123|            [web]|
|  3|    Isaac| Asimov|http://www.exampl...|2014-02-01| 456|[email, ads, web]|
|  4|  Douglas|  Adams|http://www.exampl...|2014-03-05| 321|       [ads, web]|
|  5|Arthur C.| Clarke|http://www.exampl...|2014-04-01| 213|     [email, ads]|
|  6|    Frank|Herbert|http://www.exampl...|2014-04-15| 312|     [email, web]|
|  7|Ursula K.|Le Guin|http://www.exampl...|2014-05-21| 213|            [web]|
+---+---------+-------+--------------------+----------+----+-----------------+



In [178]:
#Create df without schema difine 
blog_df_2 = spark.createDataFrame(data)

In [179]:
blog_df_2.printSchema()

root
 |-- _1: long (nullable = true)
 |-- _2: string (nullable = true)
 |-- _3: string (nullable = true)
 |-- _4: string (nullable = true)
 |-- _5: string (nullable = true)
 |-- _6: long (nullable = true)
 |-- _7: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [181]:
blogs_df.cache()

DataFrame[ID: int, First: string, Last: string, Url: string, Published: string, Hits: int, Campaigns: array<string>]

In [190]:
df2 = blogs_df.select(blogs_df['Hits']*10)

In [191]:
df2.printSchema()

root
 |-- (Hits * 10): integer (nullable = true)



In [195]:
df3= blogs_df.select((col('Hits')*10).alias('10Hits'))

In [196]:
df3.show()

+------+
|10Hits|
+------+
|  2540|
|  1230|
|  4560|
|  3210|
|  2130|
|  3120|
|  2130|
+------+



In [201]:
df4= blogs_df.select(expr('Hits * 10 ').alias('10Hits'))

In [202]:
df4.show()

+------+
|10Hits|
+------+
|  2540|
|  1230|
|  4560|
|  3210|
|  2130|
|  3120|
|  2130|
+------+



In [204]:
# 1-  Adding New COlumns Based On Specific Condition

In [205]:
blogs_df.show()

+---+---------+-------+--------------------+----------+----+-----------------+
| ID|    First|   Last|                 Url| Published|Hits|        Campaigns|
+---+---------+-------+--------------------+----------+----+-----------------+
|  1|    Jules|  Verne|http://www.exampl...|2014-01-01| 254|     [email, web]|
|  2|     H.G.|  Wells|http://www.exampl...|2014-01-02| 123|            [web]|
|  3|    Isaac| Asimov|http://www.exampl...|2014-02-01| 456|[email, ads, web]|
|  4|  Douglas|  Adams|http://www.exampl...|2014-03-05| 321|       [ads, web]|
|  5|Arthur C.| Clarke|http://www.exampl...|2014-04-01| 213|     [email, ads]|
|  6|    Frank|Herbert|http://www.exampl...|2014-04-15| 312|     [email, web]|
|  7|Ursula K.|Le Guin|http://www.exampl...|2014-05-21| 213|            [web]|
+---+---------+-------+--------------------+----------+----+-----------------+



In [208]:
df_BIgHits =blogs_df.withColumn('BigHits', col('Hits')>300)

In [211]:
df_BIgHits.show()

+---+---------+-------+--------------------+----------+----+-----------------+-------+
| ID|    First|   Last|                 Url| Published|Hits|        Campaigns|BigHits|
+---+---------+-------+--------------------+----------+----+-----------------+-------+
|  1|    Jules|  Verne|http://www.exampl...|2014-01-01| 254|     [email, web]|  false|
|  2|     H.G.|  Wells|http://www.exampl...|2014-01-02| 123|            [web]|  false|
|  3|    Isaac| Asimov|http://www.exampl...|2014-02-01| 456|[email, ads, web]|   true|
|  4|  Douglas|  Adams|http://www.exampl...|2014-03-05| 321|       [ads, web]|   true|
|  5|Arthur C.| Clarke|http://www.exampl...|2014-04-01| 213|     [email, ads]|  false|
|  6|    Frank|Herbert|http://www.exampl...|2014-04-15| 312|     [email, web]|   true|
|  7|Ursula K.|Le Guin|http://www.exampl...|2014-05-21| 213|            [web]|  false|
+---+---------+-------+--------------------+----------+----+-----------------+-------+



In [210]:
#Dispaly notebook cell with horizontal Scroll bar
from IPython.display import display , HTML
display(HTML("<style>pre {white-space: pre !important; }</style>"))

In [216]:
df_FullName =blogs_df.withColumn('Fullname', concat(col('First'), col('Last')))

In [217]:
df_FullName.show()

+---+---------+-------+--------------------+----------+----+-----------------+----------------+
| ID|    First|   Last|                 Url| Published|Hits|        Campaigns|        Fullname|
+---+---------+-------+--------------------+----------+----+-----------------+----------------+
|  1|    Jules|  Verne|http://www.exampl...|2014-01-01| 254|     [email, web]|      JulesVerne|
|  2|     H.G.|  Wells|http://www.exampl...|2014-01-02| 123|            [web]|       H.G.Wells|
|  3|    Isaac| Asimov|http://www.exampl...|2014-02-01| 456|[email, ads, web]|     IsaacAsimov|
|  4|  Douglas|  Adams|http://www.exampl...|2014-03-05| 321|       [ads, web]|    DouglasAdams|
|  5|Arthur C.| Clarke|http://www.exampl...|2014-04-01| 213|     [email, ads]| Arthur C.Clarke|
|  6|    Frank|Herbert|http://www.exampl...|2014-04-15| 312|     [email, web]|    FrankHerbert|
|  7|Ursula K.|Le Guin|http://www.exampl...|2014-05-21| 213|            [web]|Ursula K.Le Guin|
+---+---------+-------+-----------------

In [219]:
df_FullName =blogs_df.withColumn('Fullname', concat_ws(' ',col('First'), col('Last')))

In [220]:
df_FullName.show()

+---+---------+-------+--------------------+----------+----+-----------------+-----------------+
| ID|    First|   Last|                 Url| Published|Hits|        Campaigns|         Fullname|
+---+---------+-------+--------------------+----------+----+-----------------+-----------------+
|  1|    Jules|  Verne|http://www.exampl...|2014-01-01| 254|     [email, web]|      Jules Verne|
|  2|     H.G.|  Wells|http://www.exampl...|2014-01-02| 123|            [web]|       H.G. Wells|
|  3|    Isaac| Asimov|http://www.exampl...|2014-02-01| 456|[email, ads, web]|     Isaac Asimov|
|  4|  Douglas|  Adams|http://www.exampl...|2014-03-05| 321|       [ads, web]|    Douglas Adams|
|  5|Arthur C.| Clarke|http://www.exampl...|2014-04-01| 213|     [email, ads]| Arthur C. Clarke|
|  6|    Frank|Herbert|http://www.exampl...|2014-04-15| 312|     [email, web]|    Frank Herbert|
|  7|Ursula K.|Le Guin|http://www.exampl...|2014-05-21| 213|            [web]|Ursula K. Le Guin|
+---+---------+-------+-------